In [1]:
import requests
import pandas as pd
import numpy as np
import json
import config
import time
import os.path
from pathlib import Path

In [2]:
# API key
key = config.sports_key

### Define the functions required to execute the get requests and loop over the weeks of the season to build the dataframe

Get the data for each week of a season.  Save the data as a .csv file with the name being the season parameter passed to the API.

In [3]:

def get_players_by_week(season, week):
    
    """
    Function takes in two parameters:
    - Season: Input in the format '2020REG' or '2020POST' (string)
    - Week: Week of play (int)
    
    Outputs a json object of every player's stat line for the indicated week.
    """

    # Define the URL being used to access the data
    URL = f'https://api.sportsdata.io/api/nfl/fantasy/json/PlayerGameStatsByWeek/{season}/{week}'

    # Format the key for use with the URL
    # get request
    r = requests.get(URL, headers = {'Ocp-Apim-Subscription-Key': f'{key}'})
    # Convert response to JSON object
    data = r.json()

    return data

def get_scores(season, week):
    
    """
    Function takes in two parameters:
    - Season: Input in the format '2020REG' or '2020POST' (string)
    - Week: Week of play (int)
    
    Outputs a json object of game scores for the indicated week.
    """

    # Define the URL being used to access the data
    URL = f'https://api.sportsdata.io/api/nfl/odds/json/ScoresByWeek/{season}/{week}'

    # Format the key for use with the URL
    # get request
    r = requests.get(URL, headers = {'Ocp-Apim-Subscription-Key': f'{key}'})
    # Convert response to JSON object
    data = r.json()

    return data

def get_season_stats(season, weeks_to_get):
    
    """
    Take in two parameters:
    - season: Input in the format '2018REG'
    - weeks_to_get: Number of weeks of data to get.  Will always start at week 1
    
    Sends a request for player stats for each week of that season.  Converts response 
    to a dataframe using json_normalize().  Each subsequent week's dataframe is 
    appended to the first.
    """
    
    # Initialize an empty dataframe
    season_data = pd.DataFrame()
    
    # Loop over the weeks in the season, turning the response to a dataframe object
    for i in range(1, weeks_to_get + 1):
        print(f'Getting stats for week {i} of {season}.')
        
        # Store JSON object as temporary variable
        temp = get_players_by_week(season, i)
        # Convert JSON object to dataframe
        temp_df = pd.json_normalize(temp)
        # Append temp_df to season_data
        season_data = season_data.append(temp_df)
        print('Sleeping for 300 seconds')
        
        # Respect the 5 minute interval between requests
        
        time.sleep(30)
    
    # Write the data to CSV for safe keeping
    print('Done looping.  Saving dataframe to a csv file.')
    season_data.to_csv(f'Data/{season}.csv')
    print(f'.csv saved at ".../Data/{season}.csv"')
    return season_data


def get_weekly_scores(seasons, weeks_to_get):
    
    """
    Take in two parameters:
    - seasons: Input a list of seasons in the format ['2018REG', '2019REG']
    - weeks_to_get: Number of weeks of data to get.  Will always start at week 1
    
    Sends a request for scores for each week in each season listed.  Converts response 
    to a dataframe using json_normalize().  Each subsequent week's dataframe is 
    appended to the first.
    """
    # Initialize an empty datafrane
    scores = pd.DataFrame()
    
    for season in seasons:
        for i in range(1, weeks_to_get + 1):
            print(f'Getting stats for week {i} of {season}.')
            
            # Store json obhect as temporary variable
            temp = get_scores(season, i)
            # Convert json object to pandas dataframe
            temp_df = pd.json_normalize(temp)
            # Append temp_df to scores
            scores = scores.append(temp_df)
            print('Sleeping for 30 Seconds')
            
            # Respect 30s interval between requests
            
            time.sleep(30)
            
    # Write the data to CSV for safe keeping
    print('Done looping.  Saving dataframe to a csv file.')
    scores.to_csv(f'Data/weekly_scores.csv')
    print(f'.csv saved at ".../Data/weekly_scores.csv"')
    return scores

### Get Weekly Player Data

In [4]:
# If path exists, open file.
# If not, get weekly player stats for the 2018 regular season
df_2018 = Path('Data/2018REG.csv')
if df_2018.is_file():
    df_2018 = pd.read_csv(df_2018)
    print('Data loaded successfully from .csv')
else:
    print(f'Data does not exist at the current directory.')
    print('Fetching data...')
    df_2018 = get_season_stats('2018REG', 17)

Data loaded successfully from .csv


In [5]:
# If path exists, open file.
# If not, get weekly player stats for the 2019 regular season
df_2019 = Path('Data/2019REG.csv')
if df_2019.is_file():
    df_2019 = pd.read_csv(df_2019)
    print('Data loaded successfully from .csv')
else:
    print(f'Data does not exist at the current directory.')
    print('Fetching data...')
    df_2019 = get_season_stats('2019REG', 17)

Data loaded successfully from .csv


In [6]:
# If path exists, open file.
# If not, get weekly player stats for the 2020 regular season
df_2020 = Path('Data/2020REG.csv')
if df_2020.is_file():
    df_2020 = pd.read_csv(df_2020)
    print('Data loaded successfully from .csv')
else:
    print(f'Data does not exist at the current directory.')
    print('Fetching data...')
    df_2020= get_season_stats('2020REG', 17)

Data loaded successfully from .csv


In [7]:
# If path exists, open file.
# If not, get weekly player stats for the 2021 regular season
df_2021 = Path('Data/2021REG.csv')
# if df_2021.is_file():
#     df_2021 = pd.read_csv(df_2021)
#     print('Data loaded successfully from .csv')
# else:
#     print(f'Data does not exist at the current directory.')
#     print('Fetching data...')
df_2021 = get_season_stats('2021REG', 14)

Getting stats for week 1 of 2021REG.
Sleeping for 300 seconds
Getting stats for week 2 of 2021REG.
Sleeping for 300 seconds
Getting stats for week 3 of 2021REG.
Sleeping for 300 seconds
Getting stats for week 4 of 2021REG.
Sleeping for 300 seconds
Getting stats for week 5 of 2021REG.
Sleeping for 300 seconds
Getting stats for week 6 of 2021REG.
Sleeping for 300 seconds
Getting stats for week 7 of 2021REG.
Sleeping for 300 seconds
Getting stats for week 8 of 2021REG.
Sleeping for 300 seconds
Getting stats for week 9 of 2021REG.
Sleeping for 300 seconds
Getting stats for week 10 of 2021REG.
Sleeping for 300 seconds
Getting stats for week 11 of 2021REG.
Sleeping for 300 seconds
Getting stats for week 12 of 2021REG.
Sleeping for 300 seconds
Getting stats for week 13 of 2021REG.
Sleeping for 300 seconds
Getting stats for week 14 of 2021REG.
Sleeping for 300 seconds
Done looping.  Saving dataframe to a csv file.
.csv saved at ".../Data/2021REG.csv"


In [8]:
# Combine the dataframes of weekly player stats for each season.
# Write to a csv file

df_players = df_2018.append(df_2019).append(df_2020).append(df_2021)
df_players.to_csv('Data/weekly_players.csv')

### Get weekly score data

Was able to find weekly score data that included total yardage per team elsewhere, at https://www.pro-football-reference.com <br>
As such I downloaded the data from there.

In [9]:
# Load in the score data for each season and assign a value to the season column

scores_2018 = pd.read_csv('Data/2018REG_scores.csv')
scores_2018['Season'] = '2018'
scores_2019 = pd.read_csv('Data/2019REG_scores.csv')
scores_2019['Season'] = '2019'
scores_2020 = pd.read_csv('Data/2020REG_scores.csv')
scores_2020['Season'] = '2020'
scores_2021 = pd.read_csv('Data/2021REG_scores.csv')
scores_2021['Season'] = '2021'

df_scores = scores_2018.append(scores_2019).append(scores_2020).append(scores_2021)

In [10]:
def home_away(df):
    """
    Assign teams as being either the home or away team.  It also assigns the same label
    to the corresponding stats.
    
    Input:
        - df: Dataframe where we want to make the assignment
    Output:
        - Two new columns titled 'HomeTeam' and 'AwayTeam'
        """
    
    if df['Unnamed: 5'] == '@':
        df['HomeTeam'] = df['Loser/tie']
        df['HomePoints'] = df['PtsL']
        df['HomeYards'] = df['YdsL']
        df['AwayTeam'] = df['Winner/tie']
        df['AwayPoints'] = df['PtsW']
        df['AwayYards'] = df['YdsW']
    else:
        df['HomeTeam'] = df['Winner/tie']
        df['HomePoints'] = df['PtsW']
        df['HomeYards'] = df['YdsW']
        df['AwayTeam'] = df['Loser/tie']
        df['AwayPoints'] = df['PtsL']
        df['AwayYards'] = df['YdsL']
    return df

def match_string_scores(df):
    """
    Make a string that can be used to match team game data to player game data
    
    Input:
    - df: The dataframe to perform the operation on
     
    Output:
    - A string of format '20181TBNO'
        - Where Season-Week-TEAM-OPPONENT is the format
    """
    return str(df['Season']) + str(df['Week']) + str(df['Team']) + str(df['Opponent'])

### Score data needs to be formatted such that it can be merged with player data.  We want the data to reflect the offensive and defensive performances of both teams.


In [11]:
# Make a dictionary of abbreviations to full team names
abbrev_to_team = {'ARI' : 'Arizona Cardinals',
 'KC': 'Kansas City Chiefs',
 'NO': 'New Orleans Saints',
 'DET': 'Detroit Lions',
 'PHI': 'Philadelphia Eagles',
 'DAL': 'Dallas Cowboys',
 'TB': "Tampa Bay Buccaneers",
 'SEA': 'Seattle Seahawks',
 'LAR': 'Los Angeles Rams',
 'LV' : 'Las Vegas Raiders', 
 'OAK': 'Oakland Raiders',
 'HOU': 'Houston Texans',
 'SF': 'San Francisco 49ers',
 'MIN': 'Minnesota Vikings',
 'NYJ': 'New York Jets',
 'NYG': 'New York Giants',
 'CIN': 'Cincinnati Bengals',
 'DEN': 'Denver Broncos',
 'CLE': 'Cleveland Browns',
 'JAX': 'Jacksonville Jaguars',
 'CAR': 'Carolina Panthers',
 'IND': 'Indianapolis Colts',
 'BAL': 'Baltimore Ravens',
 'CHI': 'Chicago Bears',
 'BUF': 'Buffalo Bills',
 'PIT': 'Pittsburgh Steelers',
 'MIA': 'Miami Dolphins',
 'NE': 'New England Patriots',
 'LAC': 'Los Angeles Chargers',
 'TEN': 'Tennessee Titans',
 'WAS': 'Washington Football Team',
 'ATL': 'Atlanta Falcons',
 'GB': 'Green Bay Packers'}

# Make a dictionary of team name to abbreviations
team_to_abbrev = dict([(value, key) for key, value in abbrev_to_team.items()])

# Account for changes in team names over the last 4 years
team_to_abbrev['Washington Redskins'] = 'WAS'
team_to_abbrev['Oakland Raiders'] = 'LV'

In [12]:
# Apply the home away function to the dataframe

df_scores = df_scores.apply(lambda x: home_away(x), axis = 1)

# Replace the team name string with its abbreviation
df_scores['HomeTeam'].replace(team_to_abbrev, 
                               inplace = True)
df_scores['AwayTeam'].replace(team_to_abbrev,
                                inplace = True)
# df_scores['Season'] = pd.DatetimeIndex(df_scores['Date']).year


In [13]:
# Drop unnecessary columns

df_scores.drop(columns = ['Day', 
                         'PtsW', 'PtsL', 'YdsW', 'YdsL', 'TOW',
                         "TOL", 'Unnamed: 5', 'Time', 'Unnamed: 7',
                         'Winner/tie', 'Loser/tie'], 
               inplace = True)

In [14]:
# Convert this tabular data so that each row in the dataframe becomes two - one from the perspective of each team

# Design a function to get the desired data out.

team_performance = pd.DataFrame(columns = ['Season', 'Week', 'Team', 'TeamPoints', 
                                                'OpponentPoints', 'YardsFor', 'YardsAgainst'])

for row in range(len(df_scores)):
    team_performance = team_performance.append({'Season': df_scores.iloc[row]['Season'],
                                 'Week': df_scores.iloc[row]['Week'],
                                 'Team': df_scores.iloc[row]['HomeTeam'],
                                'Opponent': df_scores.iloc[row]['AwayTeam'],
                                 'TeamPoints': df_scores.iloc[row]['HomePoints'],
                                 'OpponentPoints': df_scores.iloc[row]['AwayPoints'],
                                 'YardsFor': df_scores.iloc[row]['HomeYards'],
                                 'YardsAgainst': df_scores.iloc[row]['AwayYards']}, ignore_index = True)
    team_performance = team_performance.append({'Season': df_scores.iloc[row]['Season'],
                                 'Week': df_scores.iloc[row]['Week'],
                                 'Team': df_scores.iloc[row]['AwayTeam'],
                                'Opponent': df_scores.iloc[row]['HomeTeam'],
                                 'TeamPoints': df_scores.iloc[row]['AwayPoints'],
                                 'OpponentPoints': df_scores.iloc[row]['HomePoints'],
                                 'YardsFor': df_scores.iloc[row]['AwayYards'],
                                 'YardsAgainst': df_scores.iloc[row]['HomeYards']}, ignore_index = True)

In [15]:
team_performance['MatchString'] = team_performance.apply(lambda x: match_string_scores(x), axis = 1)
team_performance

,Season,Week,Team,TeamPoints,OpponentPoints,YardsFor,YardsAgainst,Opponent,MatchString
0,2018,1,PHI,18.0,12.0,232.0,299.0,ATL,20181PHIATL
1,2018,1,ATL,12.0,18.0,299.0,232.0,PHI,20181ATLPHI
2,2018,1,NO,40.0,48.0,475.0,529.0,TB,20181NOTB
3,2018,1,TB,48.0,40.0,529.0,475.0,NO,20181TBNO
4,2018,1,BAL,47.0,3.0,369.0,153.0,BUF,20181BALBUF
...,...,...,...,...,...,...,...,...,...
2075,2021,18,KC,NaN,NaN,NaN,NaN,DEN,202118KCDEN
2076,2021,18,LAR,NaN,NaN,NaN,NaN,SF,202118LARSF
2077,2021,18,SF,NaN,NaN,NaN,NaN,LAR,202118SFLAR
2078,2021,18,ARI,NaN,NaN,NaN,NaN,SEA,202118ARISEA


In [16]:
# Save score data with team names intact to csv
team_performance.to_csv('Data/game_scores.csv')

# Drop the team and opponent columns for merging with player data
team_performance.drop(columns = ['Team', 'Opponent'],
                     inplace = True)

### Combine Score Data With Weekly Player Data

<br>
At this point there's just one more step before we're ready to merge our player data with the game data.  We need to create a 'MatchString' for the player data in the same format as the score data so that we can perform our left join on the datasets.



In [17]:
def match_string_players(df):
    
    """
    Make a string that can be used to match team game data to player game data
    
    Input:
    - df: The dataframe to perform the operation on
     
    Output:
    - A string of format '20181TBNO'
        - Where Season-Week-TEAM-OPPONENT is the format
        
    """
    
   
    return str(df['Season']) + str(df['Week']) + str(df['Team']) + str(df['Opponent'])

In [18]:
# Apply the make_match_string function to the dataset

df_players['MatchString'] = df_players.apply(lambda x: match_string_players(x), axis = 1)
df_players.head(-5)

,Unnamed: 0,GameKey,PlayerID,SeasonType,Season,GameDate,Week,Team,Opponent,HomeOrAway,...,FieldGoalsMade20to29,FieldGoalsMade30to39,FieldGoalsMade40to49,FieldGoalsMade50Plus,FantasyPointsDraftKings,InjuryStatus,TeamID,OpponentID,ScoreID,MatchString
0,0.0,201810122,8283,1,2018,2018-09-09T13:00:00,1,TB,NO,AWAY,...,0.0,0.0,0.0,0.0,45.28,NaN,33,22,16660,20181TBNO
1,1.0,201810122,18878,1,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,0.0,0.0,0.0,46.10,NaN,22,33,16660,20181NOTB
2,2.0,201810122,7242,1,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,0.0,0.0,0.0,34.56,NaN,22,33,16660,20181NOTB
3,3.0,201810110,13022,1,2018,2018-09-09T16:25:00,1,DEN,SEA,HOME,...,0.0,0.0,0.0,0.0,0.00,NaN,10,30,16664,20181DENSEA
4,4.0,201810129,18082,1,2018,2018-09-09T16:05:00,1,KC,LAC,AWAY,...,0.0,0.0,0.0,0.0,45.30,NaN,16,29,16663,20181KCLAC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1548,NaN,202111401,22721,1,2021,2021-12-13T20:15:00,14,LAR,ARI,AWAY,...,0.0,0.0,0.0,0.0,0.00,None,32,1,17889,202114LARARI
1549,NaN,202111401,22847,1,2021,2021-12-13T20:15:00,14,LAR,ARI,AWAY,...,0.0,0.0,0.0,0.0,0.00,None,32,1,17889,202114LARARI
1550,NaN,202111401,22934,1,2021,2021-12-13T20:15:00,14,LAR,ARI,AWAY,...,0.0,0.0,0.0,0.0,0.00,Out,32,1,17889,202114LARARI
1551,NaN,202111412,20089,1,2021,2021-12-12T20:20:00,14,GB,CHI,HOME,...,0.0,0.0,0.0,0.0,-0.10,None,12,6,17888,202114GBCHI


There are a bunch of player positions in the data that we have no interest in for the purposes of fantasy football.  Let's get rid of those, as well as statistics that only apply to those positions.

In [19]:
# Make a list of the positions we are interested in tracking
offensive_positions = ['WR', 'RB', 'TE', 'QB', 'K']

# Filter the dataframe to include only the players at these positions
df_players = df_players.loc[df_players.Position.isin(offensive_positions)]

# Make a list of the columns I am looking to drop
# These columns are duplicate columns or apply to defensive positions

cols_to_drop = ['SeasonType', 'Week', 'Season',
       'PositionCategory', 'Played', 'SoloTackles', 'AssistedTackles',
       'TacklesForLoss', 'Sacks', 'SackYards', 'QuarterbackHits',
       'PassesDefended', 'FumblesForced', 'FumblesRecovered',
       'FumbleReturnTouchdowns', 'Interceptions',
       'InterceptionReturnTouchdowns',  'FantasyPoints', 
       'FantasyPosition', 'PlayerGameID', 'ExtraPointsAttempted',
       'FantasyPointsFanDuel', 'FantasyPointsDraftKings', 'TeamID', 'OpponentID',
       'ScoreID']

# Drop unwanted columns from the dataset
df_players.drop(columns = cols_to_drop,
         inplace = True)

In [20]:
# Perform a left join on data and df_scores

data = df_players.merge(team_performance,
                 how = 'left',
                 left_on = 'MatchString',
                 right_on = 'MatchString')

In [21]:
data.columns

Index(['Unnamed: 0', 'GameKey', 'PlayerID', 'GameDate', 'Team', 'Opponent',
       'HomeOrAway', 'Number', 'Name', 'Position', 'Started',
       'PassingAttempts', 'PassingCompletions', 'PassingYards',
       'PassingCompletionPercentage', 'PassingYardsPerAttempt',
       'PassingYardsPerCompletion', 'PassingTouchdowns',
       'PassingInterceptions', 'PassingRating', 'PassingLong', 'PassingSacks',
       'PassingSackYards', 'RushingAttempts', 'RushingYards',
       'RushingYardsPerAttempt', 'RushingTouchdowns', 'RushingLong',
       'ReceivingTargets', 'Receptions', 'ReceivingYards',
       'ReceivingYardsPerReception', 'ReceivingTouchdowns', 'ReceivingLong',
       'Fumbles', 'FumblesLost', 'PuntReturns', 'PuntReturnYards',
       'PuntReturnTouchdowns', 'KickReturns', 'KickReturnYards',
       'KickReturnTouchdowns', 'FieldGoalsAttempted', 'FieldGoalsMade',
       'ExtraPointsMade', 'TwoPointConversionPasses', 'TwoPointConversionRuns',
       'TwoPointConversionReceptions', 'Fantasy

In [22]:
# Cast the Week column to type string

data['Week'] = data['Week'].astype(str)

In [23]:
# Let's inspect the data to make sure nothing went haywire
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33508 entries, 0 to 33507
Data columns (total 62 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    26551 non-null  float64
 1   GameKey                       33508 non-null  object 
 2   PlayerID                      33508 non-null  int64  
 3   GameDate                      33508 non-null  object 
 4   Team                          33508 non-null  object 
 5   Opponent                      33508 non-null  object 
 6   HomeOrAway                    33508 non-null  object 
 7   Number                        33508 non-null  int64  
 8   Name                          33508 non-null  object 
 9   Position                      33508 non-null  object 
 10  Started                       33508 non-null  int64  
 11  PassingAttempts               33508 non-null  float64
 12  PassingCompletions            33508 non-null  float64
 13  P

In [24]:
# Convert week and season datatypes back to int
data['Week'] = data['Week'].astype(int)
data['Season'] = data['Season'].astype(int)

In [25]:
data.head()

,Unnamed: 0,GameKey,PlayerID,GameDate,Team,Opponent,HomeOrAway,Number,Name,Position,...,FieldGoalsMade40to49,FieldGoalsMade50Plus,InjuryStatus,MatchString,Season,Week,TeamPoints,OpponentPoints,YardsFor,YardsAgainst
0,0.0,201810122,8283,2018-09-09T13:00:00,TB,NO,AWAY,14,Ryan Fitzpatrick,QB,...,0.0,0.0,NaN,20181TBNO,2018,1,48.0,40.0,529.0,475.0
1,1.0,201810122,18878,2018-09-09T13:00:00,NO,TB,HOME,41,Alvin Kamara,RB,...,0.0,0.0,NaN,20181NOTB,2018,1,40.0,48.0,475.0,529.0
2,2.0,201810122,7242,2018-09-09T13:00:00,NO,TB,HOME,9,Drew Brees,QB,...,0.0,0.0,NaN,20181NOTB,2018,1,40.0,48.0,475.0,529.0
3,4.0,201810129,18082,2018-09-09T16:05:00,KC,LAC,AWAY,10,Tyreek Hill,WR,...,0.0,0.0,NaN,20181KCLAC,2018,1,38.0,28.0,362.0,541.0
4,5.0,201810108,18983,2018-09-09T13:00:00,PIT,CLE,AWAY,30,James Conner,RB,...,0.0,0.0,NaN,20181PITCLE,2018,1,21.0,21.0,472.0,327.0


In [26]:
# Drop data where the target variable is null
data.dropna(subset =['FantasyPointsPPR'],
           inplace = True)

# Write the combined score and player data to a csv file

data.to_csv('Data/weekly_data.csv')